<a href="https://colab.research.google.com/github/YuJessica/113Dcapstone/blob/main/PreprocessImages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Import Overarching Libraries and Initialize Global Variables ###

import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
from skimage.transform import resize
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image, ImageOps
import cv2
from sklearn.utils import class_weight, shuffle
from keras.losses import binary_crossentropy
#from keras.applications.resnet50 import preprocess_input
import keras.backend as K
import tensorflow as tf
from sklearn.metrics import f1_score, fbeta_score
from keras.utils import Sequence
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

WORKERS = 2
CHANNEL = 3

import warnings
warnings.filterwarnings("ignore")
IMG_SIZE = 512
NUM_CLASSES = 5
SEED = 77
TRAIN_NUM = -1 # use 1000 when you just want to explore new idea, use -1 for full train

In [ ]:
!pip install -q opendatasets

import opendatasets as od

od.download('https://www.kaggle.com/datasets/tanlikesmath/diabetic-retinopathy-resized') # insert kaggle username and key
# User: ranavarro2
# Key: 25048ecc59000cfce78650325beafc76

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: ranavarro2
Your Kaggle Key: ··········


100%|██████████| 7.25G/7.25G [04:18<00:00, 30.2MB/s]


In [ ]:
if not os.path.exists('diabetic-retinopathy-resized/processed'):
  os.mkdir('diabetic-retinopathy-resized/processed') # Create a directory to hold processed image
if not os.path.exists('diabetic-retinopathy-resized/processed/0'):
  os.mkdir('diabetic-retinopathy-resized/processed/0')
if not os.path.exists('diabetic-retinopathy-resized/processed/1'):
  os.mkdir('diabetic-retinopathy-resized/processed/1')
if not os.path.exists('diabetic-retinopathy-resized/processed/2'):
  os.mkdir('diabetic-retinopathy-resized/processed/2')
if not os.path.exists('diabetic-retinopathy-resized/processed/3'):
  os.mkdir('diabetic-retinopathy-resized/processed/3')
if not os.path.exists('diabetic-retinopathy-resized/processed/4'):
  os.mkdir('diabetic-retinopathy-resized/processed/4')


In [ ]:
base_image_dir = os.path.join('diabetic-retinopathy-resized/resized_train/resized_train/')
retina_df = pd.read_csv(os.path.join('diabetic-retinopathy-resized/trainLabels.csv'))
retina_df['PatientId'] = retina_df['image'].map(lambda x: x.split('_')[0])
retina_df['path'] = retina_df['image'].map(lambda x: os.path.join(base_image_dir,
                                                         '{}.jpeg'.format(x)))
retina_df['exists'] = retina_df['path'].map(os.path.exists)
print(retina_df['exists'].sum(), 'images found of', retina_df.shape[0], 'total')
retina_df['eye'] = retina_df['image'].map(lambda x: 1 if x.split('_')[-1]=='left' else 0)
from keras import utils
retina_df['level_cat'] = retina_df['level'].map(lambda x: utils.to_categorical(x, 1+retina_df['level'].max()))

retina_df.dropna(inplace = True)
retina_df = retina_df[retina_df['exists']]
retina_df.sample(3)

35126 images found of 35126 total


,image,level,PatientId,path,exists,eye,level_cat
6405,8032_right,0,8032,diabetic-retinopathy-resized/resized_train/res...,True,0,"[1.0, 0.0, 0.0, 0.0, 0.0]"
4598,5791_left,0,5791,diabetic-retinopathy-resized/resized_train/res...,True,1,"[1.0, 0.0, 0.0, 0.0, 0.0]"
11941,14986_right,0,14986,diabetic-retinopathy-resized/resized_train/res...,True,0,"[1.0, 0.0, 0.0, 0.0, 0.0]"


In [ ]:
### Auxilary Functions used to preprocess images

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol

        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img

def load_ben_color(path, sigmaX=10):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
    return image

In [ ]:
for id in range(np.size(retina_df.image)):
  imgPath = retina_df.path[id]
  imgPost = load_ben_color(imgPath) # Preprocess image
  newPath = f"diabetic-retinopathy-resized/processed/{retina_df.level[id]}/{retina_df.image[id]}.jpeg"
  if not os.path.exists(newPath):
    cv2.imwrite(newPath, imgPost)   # Save preprocessed images to another directory ...
    retina_df.path[id]= newPath      # ... and reroute the path of the corresponding image to the new directory

In [ ]:
!zip -r 'drive/MyDrive/processedSet2.zip' 'diabetic-retinopathy-resized/processed'

Streaming output truncated to the last 5000 lines.
  adding: diabetic-retinopathy-resized/processed/0/38928_right.jpeg (deflated 0%)
  adding: diabetic-retinopathy-resized/processed/0/27986_left.jpeg (deflated 0%)
  adding: diabetic-retinopathy-resized/processed/0/19702_left.jpeg (deflated 0%)
  adding: diabetic-retinopathy-resized/processed/0/29879_left.jpeg (deflated 0%)
  adding: diabetic-retinopathy-resized/processed/0/12729_left.jpeg (deflated 0%)
  adding: diabetic-retinopathy-resized/processed/0/8657_right.jpeg (deflated 0%)
  adding: diabetic-retinopathy-resized/processed/0/43138_right.jpeg (deflated 0%)
  adding: diabetic-retinopathy-resized/processed/0/40780_right.jpeg (deflated 0%)
  adding: diabetic-retinopathy-resized/processed/0/33742_left.jpeg (deflated 0%)
  adding: diabetic-retinopathy-resized/processed/0/616_right.jpeg (deflated 0%)
  adding: diabetic-retinopathy-resized/processed/0/11888_right.jpeg (deflated 0%)
  adding: diabetic-retinopathy-resized/processed/0/2063

In [ ]:
import zipfile
def zip_and_remove(path):
    ziph = zipfile.ZipFile(f'{path}.zip', 'w', zipfile.ZIP_DEFLATED)

    for root, dirs, files in os.walk(path):
        for file in tqdm(files):
            file_path = os.path.join(root, file)
            ziph.write(file_path)
            os.remove(file_path)

    ziph.close()

In [ ]:
zip_and_remove('diabetic-retinopathy-resized/processed')

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
